In [ ]:
# import stackstac
# import pystac_client

In [ ]:
# URL = "https://landsatlook.usgs.gov/stac-server"
# catalog = pystac_client.Client.open(URL)

# stac_items = catalog.search(
#     intersects=dict(type="Point", coordinates=[geom.bounds[2], geom.bounds[3]]),
#     collections='landsat-c2l1',
#     datetime=timeRange
# ).get_all_items()

# scene = stackstac.stack(stac_items)
# scene

In [ ]:
%matplotlib inline
import satsearch
print(satsearch.__version__)
# print(satsearch.config.API_URL)
from satsearch import Search
import geopandas as gpd
import ast
import pandas as pd
import geoviews as gv
import hvplot.pandas
from ipywidgets import interact
from IPython.display import display, Image
import intake # if you've installed intake-STAC, it will automatically import alongside intake
import xarray as xr
import matplotlib.pyplot as plt
import boto3
import rasterio as rio
from rasterio.session import AWSSession
from rasterio.plot import show
from dask.utils import SerializableLock
import os
import hvplot.xarray
import numpy as np
from pyproj import Proj, transform

# Suppress library deprecation warnings
import warnings
warnings.filterwarnings('ignore')

### Search for Landsat imagery

To explore and access COG's easily we will use a SpatioTemporal Asset Catalog (STAC). The STAC provides a common metadata format to make it easier to index and querrying S3 buckets for geospatial data. Learn more here: https://github.com/radiantearth/stac-spec.

In [ ]:
# Sets up credentials for acquiring images through dask/xarray
os.environ["AWS_REQUEST_PAYER"] = "requester"

# Sets up proper credentials for acquiring data through rasterio
aws_session = AWSSession(boto3.Session(), requester_pays=True)

Extract geometry bounds from the ICESat-2 KML file used above so that we can perform the Landsat spatial search.

In [ ]:
# Extract geometry bounds
geom = jk.geometry[0]
print(geom.bounds)

We will search for imagery in STAC catalog using satsearch: 
https://github.com/sat-utils/sat-search

In [ ]:
# Search STAC API for Landsat images based on a bounding box, date and other metadata if desired

bbox = (geom.bounds[0], geom.bounds[1], geom.bounds[2], geom.bounds[3]) #(west, south, east, north) 

timeRange = '2019-05-06/2019-05-07'
    
results = Search.search(url='https://landsatlook.usgs.gov/stac-server',
                        collection='usgs-landsat/collection02/',
                        datetime=timeRange,
                        bbox=bbox,    
                        # properties=properties,
                        sort=['<datetime'])

print('%s items' % results.found())
items = results.items()

# Save search to geojson file
gjson_outfile = f'/home/jovyan/website2022/book/tutorials/DataIntegration/Landsat.geojson'
items.save(gjson_outfile)

In [ ]:
# Search for Landsat or Sentinel images based on a bounding box, date and other metadata if desired
# Save to geojson file
# NOTE this STAC API endpoint does not currently search the entire catalog

satellite = 'Landsat8'

bbox = (-105.30, -74.970, -105.08, -74.920) #(west, south, east, north) 

timeRange = '2013-10-12/2013-11-18'

if satellite=='Landsat8':
    # url = 'https://ibhoyw8md9.execute-api.us-west-2.amazonaws.com/prod'
    url = 'https://landsatlook.usgs.gov/stac-server'
    collection = 'usgs-landsat/collection02/'
    band = 'blue'
    colnm = ['landsat:wrs_path','landsat:wrs_row']
    end = '_T2' # end of file name if don't want all
    qa_band = 'qa_pixel'
elif satellite=='Sentinel2':
    url = 'https://earth-search.aws.element84.com/v0' # maybe also https://services.sentinel-hub.com/api/v1/catalog/
    collection = 'sentinel-s2-l2a-cogs'
    band = 'B02'
    colnm = ['sentinel:latitude_band','sentinel:grid_square']
    end = 'L2A'
    qa_band = 'SCL'
elif satellite=='Sentinel1': ### Doesn't work
    url = 'http://eocloud.sentinel-hub.com/search'
    collection = 'sentinel-s1-rtc-indigo'
    
results = Search.search(url=url,
                        collection=collection,
                        datetime=timeRange,
                        bbox=bbox,    
                        # properties=properties,
                        sort=['<datetime'])

print('%s items' % results.found())
items = results.items()
items.save(f'/home/jovyan/shared-readwrite/geostacks/polynyas/{satellite}.geojson')

We can include property searches, such as path, row, cloud-cover, as well with the `properties` flag.

We are given a satsearch collection of items (images)

In [ ]:
items

Load the geojson file into geopandas and inspect the items we want to collect

In [ ]:
# Load the geojson file
gf = gpd.read_file(gjson_outfile)
gf.head(2)

In [ ]:
# Plot search area of interest and frames on a map using Holoviz Libraries (more on these later)
cols = gf.loc[:,('id','landsat:wrs_path','landsat:wrs_row','geometry')]
footprints = cols.hvplot(geo=True, line_color='k', hover_cols=['landsat:wrs_path','landsat:wrs_row'], alpha=0.2, title='Landsat 8 T1',tiles='ESRI')
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * labels

### Intake all scenes using the intake-STAC library
`Intake-STAC` facilitates discovering, exploring, and loading spatio-temporal datasets by providing Intake Drivers for STAC catalogs. This provides a simple toolkit for working with STAC catalogs and for loading STAC assets as `xarray` objects.

In [ ]:
catalog = intake.open_stac_item_collection(items)
list(catalog)

Let's explore the metadata and keys for the first scene

In [ ]:
sceneids = list(catalog)
item3 = catalog[sceneids[3]]
item3.metadata
# for keys in item3.keys():
    # print (keys)

We can explore the metadata for any of these:

In [ ]:
item3['blue'].metadata

In [ ]:
# This is the url needed to grab data from the S3 bucket
# From the satsearch catalog:
items[3].asset('blue')['alternate']['s3']['href'] # can use item asset name or title (blue or B2)

# From the intake-STAC catalog
item3.blue.metadata['alternate']['s3']['href'] # must use item asset name (blue)

### Open and visualize each image using RasterIO 

In [ ]:
import rasterio as rio

# Retrieve first scene using rio
item_url = item3.blue.metadata['alternate']['s3']['href']

# Read and plot with grid coordinates 
with rio.Env(aws_session):
    with rio.open(item_url) as src:
        fig, ax = plt.subplots(figsize=(9,8))
        
        # To plot
        show(src,1)
        
        # To open data into a numpy array
        profile = src.profile
        arr = src.read(1)

## Read into Xarray

In [ ]:
def retrieve_bands(item,satellite):
    '''Get band names for specific satellite
    input:
        item = stac catalog item
        satellite = str satellite name
    return:
        band_names = list of str of desired band names
    '''
    
    band_names = []
    if satellite=='Landsat8':
        for k in item.keys():
            M = getattr(item, k).metadata
            if 'eo:bands' in M:
                resol = M['eo:bands'][0]['gsd']
#                 print(k, resol)
                if resol >= 30: # thermal bands are up sampled from 100 to 30
                    band_names.append(k)
        
    elif satellite=='Sentinel2': # no thermal data so therm will be empty
        for k in item.keys():
            try:
                M = getattr(item, k).metadata
                if 'data' in M['roles']:
                    if 'eo:bands' in M:
                        resol = M['gsd']
                        # print(k, resol)
                        if resol == 10:
                            band_names.append(k)
            except:
                continue
                
    return band_names

def add_mask(qa_band,satellite):
    ''' Add the qa band for the satellite
    
    input:
        qa_band = str name of satellite qa band
        satellite = str name of satellite
    
    return: 
        mask_c = mask array or qa band
    '''
    
    if satellite == 'Landsat8':
        # Create and add cloud mask (this mask is technically everything except nan,ocean,ice clear sky)
        # 1 is no data, 21952 is ocean, 30048 is ice (can change this to add more)
        # Access mask via 'ts_scenes.isel(time=0).mask'
        
        # Determined qa_pixel values
        ice = [30048, 30304]
        ocean = [21952, 22208]
        cloud = [21762,22018,22280,23826,24082,29986,30242,54790,55052,56598,56660,56854,56916,62758,62820,63014,63076]
        nans = [1]
        
        qa = scene.sel(band=qa_band)
        cond = np.logical_or((qa.isin(ocean))|(qa.isin(ice)), qa==1)
#         cond = np.logical_or((qa==21952)|(qa==30048), qa==1)
        qa_c = qa.where(~cond, np.nan)
        cond = np.logical_or((qa_c.isin(ocean))|(qa_c.isin(ice)), qa_c==1)
        mask_c = qa_c.where(qa_c.isnull(),2) # cloud = 2
        
        # For optional ice mask
        qa_io = qa.where(qa.isin(ice), np.nan)
        mask_io = qa_io.where(~(qa_io.isin(ice)),1) # ice = 1
        mask_c = mask_io.where(mask_io==1,mask_c)
    
    elif satellite == 'Sentinel2':
        mask_c = scene.sel(band=qa_band)
    
    return mask_c

def time_index_from_filenames(file_links):
    '''
    Helper function to create a pandas DatetimeIndex ***unusued currently
    '''
    return [datetime.strptime(f.split('.')[-5], '%Y%jT%H%M%S') for f in file_links]

In [ ]:
# band_names = []

# # Get band names
# sceneid = catalog[sceneids[1]]
# for k in sceneid.keys():
#     M = getattr(sceneid, k).metadata
#     if 'eo:bands' in M:
#         resol = M['eo:bands'][0]['gsd']
#         print(k, resol)
#         if resol == 30: 
#             band_names.append(k)
            
# # Add qa band
# band_names.append('qa_pixel')

And now open all of it...

In [ ]:
# # Import to xarray
# # In xarray dataframe nans are in locations where concat of multiple scenes has expanded the grid (i.e. different path/rows).
# scenes = []

# for sceneid in sceneids:
#     sceneid = catalog[sceneid]
#     print (sceneid.name)

#     bands = []

#     # Construct xarray for scene, open each band, append and concatenate together to create a scene, 
#     # then append and concatenate each scene to create the full dataframe 
#     for band_name in band_names:
#         band = sceneid[band_name](chunks=dict(band=1, x=2048, y=2048),urlpath=sceneid[band_name].metadata['alternate']['s3']['href']).to_dask()
#         band['band'] = [band_name]
#         bands.append(band)
#     scene = xr.concat(bands, dim='band')
#     scenes.append(scene)

# # Concatenate scenes with time variable
# ls_scenes = xr.concat(scenes, dim=time_var)

# ls_scenes

In [ ]:
# Import to xarray with cloud mask  ***Creating cloud mask is slow
# nans are in locations where concat of multiple scenes has expanded the grid
# Would like to use SR for all bands except TIR, but they are not available for many scenes so would
# need a separate model for scenes with SR and those without.
scenes = []

# Create time variable for time dim
time_var = xr.Variable('time',gf.loc[gf.id.isin([item for item in sceneid if item.endswith(end)])]['datetime'])
_, index = np.unique(time_var, return_index=True) # for some reason there are duplicates in Sentinel
time_var = time_var[index]
    
for item in sceneid:
    if item.endswith(end):
        item = catalog[item]
        print (item.name)
        
        bands = []
        
        # Get desired band names, currently hard coded for resolutions
        band_names = retrieve_bands(item,satellite)
                    
        # Add qa band
        band_names.append(qa_band)

        # Concatenate desired bands and construct xarray for scene
        for band_name in band_names:
            if satellite=='Landsat8':
                url = item[band_name].metadata['alternate']['s3']['href']
            elif satellite=='Sentinel2':
                url = item[band_name].metadata['href']
            band = item[band_name](chunks=dict(band=1, x=2048, y=2048),urlpath=url).to_dask() # Specify chunk size, landsat is prob in 512 chunks so used multiple
            band['band'] = [band_name]
            bands.append(band)
        scene = xr.concat(bands, dim='band')
        
        # Add qa band or mask and concatenate, in L8 cloud=2/ice=1
        mask = add_mask(qa_band,satellite)
        scene.coords['mask'] = (('y', 'x'), mask.data)
        scenes.append(scene)

# Concatenate scenes with time variable ***This is the slowest
ts_scenes = xr.concat(scenes, dim=time_var)

# Get epsg for Landsat images
# epsg = item.metadata['proj:epsg']
# epsg = ts_scenes.crs
pix = ts_scenes.transform[0]

ts_scenes

We now have 2 Landsat scenes with all of the bands we are interested in stored in an `xarray`, but you can imagine that this exact code can scale to years worth of data and bands.

`Stack-STAC` is another package that works similarly and may be more efficient for searching and reading in data from a STAC catalog. However, we will not show it here because it may not be as flexible for reading in selected data and isn't as transparent in how it builds a `DataArray` (https://github.com/gjoseph92/stackstac). 

From here, we easily subset one image at a time or the entire `xarray`:

In [ ]:
sbands = ['blue', 'nir08', 'swir16']

# Select the first datetime
t = ls_scenes.time.values[1]
print (t)

# Upper left and lower right coordinates for subsetting to Jakobshavn area
ulx = 300000
uly = 7695000
lrx = 330000
lry = 7670000

image = ls_scenes.sel(time=t,band=sbands,y=slice(lry,uly),x=slice(ulx,lrx))

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
image.sel(band='blue').plot()

Since this data is in UTM 22N, we can reproject to the standard lat/lon coordinate system (WGS-84) and map with the ICESat-2 and ATM lines. However, we realized the CRS data was inaccurate in the DataArray in comparison to the STAC catalog...

In [ ]:
print (item3.metadata['proj:epsg'])
print (image.rio.crs)

The inaccurate CRS lableling is something that may arise with some Landsat UTMs so if you plot your data after reprojecting and it looks wild, double check that you started with the correct UTM information.

We re-label our CRS info in our `DataArray` to correct this:

In [ ]:
image = image.rio.write_crs(32623)
image.rio.crs

Now we can reproject and map!!!

In [ ]:
image = image.rio.reproject(4326)

ISlats = [is2_gt2r['lat'].min(), is2_gt2r['lat'].max()]
ISlons = (is2_gt2r['lon'].max(), is2_gt2r['lon'].min())

ATMlats = [atm_l2['Latitude(deg)'].min(), atm_l2['Latitude(deg)'].max()]
ATMlons = [atm_l2['Longitude(deg)'].max(), atm_l2['Longitude(deg)'].min()]

fig, ax = plt.subplots(figsize=(8,6))
image.sel(band='blue').plot()
plt.plot(ISlons,ISlats,color = 'green')
plt.plot(ATMlons,ATMlats,color = 'orange')